<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/11_mlp_vs_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook 11: MLP vs CNN

🔹 Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


🔹 Cargar datos preprocesados

In [ ]:
processed_dir = "../data/processed/"
x_train = np.load(processed_dir + "x_train.npy")
y_train = np.load(processed_dir + "y_train.npy")
x_test  = np.load(processed_dir + "x_test.npy")
y_test  = np.load(processed_dir + "y_test.npy")

🔹 Aplanar imágenes para el MLP

In [ ]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat  = x_test.reshape(x_test.shape[0], -1)

🔹 Construir MLP simple

In [ ]:
mlp = Sequential([
    Dense(512, activation='relu', input_shape=(x_train_flat.shape[1],)),
    Dense(256, activation='relu'),
    Dense(10, activation='softmax')
])

mlp.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
mlp_history = mlp.fit(x_train_flat, y_train, validation_data=(x_test_flat, y_test), epochs=20, batch_size=64)

🔹 Construir CNN simple (para comparación)

In [ ]:
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

cnn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_history = cnn.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=64)

🔹 Comparar curvas de aprendizaje

In [ ]:
plt.figure(figsize=(12,5))

# Accuracy

In [ ]:
plt.subplot(1,2,1)
plt.plot(mlp_history.history['val_accuracy'], label='MLP Val Accuracy')
plt.plot(cnn_history.history['val_accuracy'], label='CNN Val Accuracy')
plt.title("Comparativa de Accuracy")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")
plt.legend()


# Loss

In [ ]:
plt.subplot(1,2,2)
plt.plot(mlp_history.history['val_loss'], label='MLP Val Loss')
plt.plot(cnn_history.history['val_loss'], label='CNN Val Loss')
plt.title("Comparativa de Loss")
plt.xlabel("Épocas")
plt.ylabel("Loss")
plt.legend()

plt.show()


🔹 Matriz de confusión de CNN

In [ ]:
y_pred_cnn = np.argmax(cnn.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred_cnn)
disp = ConfusionMatrixDisplay(cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix CNN")
plt.show()